### Quantization aware training

#### Apply quantization aware training to whole pre-trained baseline model and check summary. All layers are now prefixed by "quant"

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from model import LinearSpecLayer

# load baseline keras model which still has the activation layer in the end
keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activation/"
keras_baselineModel_activation_model = keras.models.load_model(keras_baselineModel_activation_path)

annotated_model = tf.keras.models.clone_model(keras_baselineModel_activation_model)

# Register the custom layer with TensorFlow
#tf.keras.utils.get_custom_objects()['LinearSpecLayer'] = LinearSpecLayer
#tf.keras.utils.get_custom_objects()['GlobalLogExpPooling2D'] = GlobalLogExpPooling2D

# Apply quantization using `quantize_apply`
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

quant_aware_model.summary()

2023-08-09 14:42:39.665954: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 14:42:39.709780: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 14:42:39.710356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 14:42:40.461894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TypeError: Error when deserializing class 'LinearSpecLayer' using config={'name': 'ADVANCED_SPEC1', 'trainable': True, 'dtype': 'float32', 'data_format': 'channels_last', 'sample_rate': 48000, 'spec_shape': [64, 512], 'frame_step': 280, 'fmin': 150, 'fmax': 15000, 'frame_length': 512}.

Exception encountered: ('Keyword argument not understood:', 'data_format')

#### Does not work because of custom layer. Try to just quantize certain layers such as Conv2D for istance:

In [15]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from model import LinearSpecLayer

# load baseline keras model which still has the activation layer in the end
keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activation/"
keras_baselineModel_activation_model = keras.models.load_model(keras_baselineModel_activation_path)

def apply_quantization_to_conv(layer):
  if isinstance(layer, tf.keras.layers.Conv2D):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    keras_baselineModel_activation_model,
    clone_function=apply_quantization_to_conv,
)

# Apply quantization using `quantize_apply`
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

#quant_aware_model.summary()

ValueError: Unable to clone model. This generally happens if you used custom Keras layers or objects in your model. Please specify them via `quantize_scope` for your calls to `quantize_model` and `quantize_apply`. [Error when deserializing class 'LinearSpecLayer' using config={'name': 'ADVANCED_SPEC1', 'trainable': True, 'dtype': 'float32', 'data_format': 'channels_last', 'sample_rate': 48000, 'spec_shape': [64, 512], 'frame_step': 280, 'fmin': 150, 'fmax': 15000, 'frame_length': 512}.

Exception encountered: ('Keyword argument not understood:', 'data_format')].

#### As it is during Clone, also this filtert Quantization is not possible. We add custom code for the layers:

In [14]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from model import LinearSpecLayer, GlobalLogExpPooling2D

keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activation/"
keras_baselineModel_activation_model = keras.models.load_model(keras_baselineModel_activation_path)

quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

### just custom layers
# def apply_quantization_to_custom_layers(layer):
#     if type(layer).__name__ == 'LinearSpecLayer':
#         return quantize_annotate_layer(layer)
#     elif type(layer).__name__ == 'GlobalLogExpPooling2D':
#         return quantize_annotate_layer(layer)
#     return layer

# annotated_model = tf.keras.models.clone_model(
#     keras_baselineModel_activation_model,
#     clone_function=apply_quantization_to_custom_layers,
# )
###


### just conv2D layers
def apply_quantization_to_conv(layer):
  if isinstance(layer, tf.keras.layers.Conv2D):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    keras_baselineModel_activation_model,
    clone_function=apply_quantization_to_conv,
)
###

### all Layers
# def annotate_all_layers(layer):
#     return quantize_annotate_layer(layer)

# annotated_model_debug = tf.keras.models.clone_model(
#     keras_baselineModel_activation_model,
#     clone_function=annotate_all_layers,
# )
###


with quantize_scope(
    {'LinearSpecLayer': LinearSpecLayer,
    'GlobalLogExpPooling2D': GlobalLogExpPooling2D
    }):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model_debug)

final spec shape:Tensor("quantize_annotate_1312/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)


RuntimeError: Layer ADVANCED_SPEC1:<class 'model.LinearSpecLayer'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.

##### No matter if we just use the custom layers, all layers, or only Conv layers and additionally given own layer code, own tfmot.quantization.keras.QuantizeConfig seems to be necessary.

In [7]:
for layer in annotated_model.layers:
    print(f"Layer: {type(layer).__name__}, Class: {layer.__class__}")

Layer: InputLayer, Class: <class 'keras.src.engine.input_layer.InputLayer'>
Layer: QuantizeAnnotate, Class: <class 'tensorflow_model_optimization.python.core.quantization.keras.quantize_annotate.QuantizeAnnotate'>
Layer: BatchNormalization, Class: <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'>
Layer: Conv2D, Class: <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
Layer: BatchNormalization, Class: <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'>
Layer: Activation, Class: <class 'keras.src.layers.core.activation.Activation'>
Layer: MaxPooling2D, Class: <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>
Layer: AveragePooling2D, Class: <class 'keras.src.layers.pooling.average_pooling2d.AveragePooling2D'>
Layer: Concatenate, Class: <class 'keras.src.layers.merging.concatenate.Concatenate'>
Layer: Activation, Class: <class 'keras.src.layers.core.activation.Activation'>
Layer: Conv2D, Class: <class 'keras.src.lay